# [PyMySQL](https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html)
### 설치
```shell
$(.venv) pip install PyMySQL
```

### MySQL Connection 연결 
- [RuntimeError 발생 시](https://stackoverflow.com/questions/54477829/cryptography-is-required-for-sha256-password-or-caching-sha2-password)

connect() 함수를 이용하면 MySQL host내 DB와 직접 연결할 수 있습니다.
- user: user name
- passwd: 설정한 패스워드
- host: DB가 존재하는 host
- db: 연결할 데이터베이스 이름
- charset: 인코딩 설정

In [1]:
import pymysql 

In [2]:
con = pymysql.connect(host='localhost', user='urstory', password='u1234',
                        db='examplesdb', charset='utf8') # 한글처리 (charset = 'utf8')


In [3]:
cur = con.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = "show tables"
cur.execute(sql)

1

In [5]:
# 데이타 Fetch
rows = cur.fetchall()
print(rows)     # 전체 rows

[{'Tables_in_examplesdb': 'board_table'}]


In [7]:
import pandas as pd 

pd.DataFrame(rows)

,Tables_in_examplesdb
0,board_table


## Select 

In [8]:
sql = "select * from board_table"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,num,title,writer,content,reg_date,cnt
0,1,title1,Writer1,Contents,2023-11-13 08:28:19,0


## Insert

In [9]:
sql = '''INSERT INTO board_table (title, writer, content) 
    VALUES ('title2', 'writer2', 'content2');'''

cur.execute(sql)
con.commit()

In [10]:
sql = "select * from board_table"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,num,title,writer,content,reg_date,cnt
0,1,title1,Writer1,Contents,2023-11-13 08:28:19,0
1,2,title2,writer2,content2,2023-11-14 05:28:03,0


## Update

In [11]:
sql = '''
UPDATE board_table
SET cnt = 3
WHERE num = 2;
'''

cur.execute(sql)
con.commit()

In [12]:
sql = "select * from board_table"
cur.execute(sql)

rows = cur.fetchall()
pd.DataFrame(rows)

,num,title,writer,content,reg_date,cnt
0,1,title1,Writer1,Contents,2023-11-13 08:28:19,0
1,2,title2,writer2,content2,2023-11-14 05:28:03,3


# Placeholder 
> 만약 DB내 데이터에 대해 대량 삽입/변경/삭제가 필요한데, 조건이 다 다르다면? Placeholder를 이용할 수 있습니다!

Placeholder의 특징은,
- 두번째 파라미터에 들어간 데이터 순서대로 SQL이 적용되고,
- 특히 문자의 경우 따옴표 등의 특수문자들이 자동으로 이스케이프(Escape)되어 처리됩니다. (완전 간편!)
- 문자열, 숫자 등에 관계 없이 대치할 값은 모두 `%s`로 쓰입니다. (일반 문자열에서 사용하는 `%s`, `%d`와는 다름)
- `%s`는 컬럼 값을 대치할 때만 사용할 수 있습니다.

## Insert

In [13]:
a_data = ['title3', 'writer3', 'content3']

sql = '''
INSERT INTO board_table (title, writer, content) 
VALUES (%s, %s, %s);
'''

cur.execute(sql, a_data)
con.commit()

## Select

In [17]:
sql = """
select * from board_table where num >= %s;
"""

cur.execute(sql, 2)

rows = cur.fetchall()
pd.DataFrame(rows)

,num,title,writer,content,reg_date,cnt
0,2,title2,writer2,content2,2023-11-14 05:28:03,3
1,3,title3,writer3,content3,2023-11-14 05:33:20,0


## executemany

In [18]:
datas = [
    ['title4', 'writer4', 'content4'],
    ['title5', 'writer5', 'content5'],
    ['title6', 'writer6', 'content6']
]

sql = '''
INSERT INTO board_table (title, writer, content) 
VALUES (%s, %s, %s);
'''

cur.executemany(sql, datas)
con.commit()

In [19]:
sql = """
select * from board_table where num >= %s;
"""

cur.execute(sql, 2)

rows = cur.fetchall()
pd.DataFrame(rows)

,num,title,writer,content,reg_date,cnt
0,2,title2,writer2,content2,2023-11-14 05:28:03,3
1,3,title3,writer3,content3,2023-11-14 05:33:20,0
2,4,title4,writer4,content4,2023-11-14 05:36:44,0
3,5,title5,writer5,content5,2023-11-14 05:36:44,0
4,6,title6,writer6,content6,2023-11-14 05:36:44,0


# Database Session Close

In [20]:
con.close()

# With db.py

In [22]:
import db

In [24]:
sql = """
select * from board_table where num >= %s;
"""
params = 3

results = db.select_datas(sql, params)
pd.DataFrame(results)

,num,title,writer,content,reg_date,cnt
0,3,title3,writer3,content3,2023-11-14 05:33:20,0
1,4,title4,writer4,content4,2023-11-14 05:36:44,0
2,5,title5,writer5,content5,2023-11-14 05:36:44,0
3,6,title6,writer6,content6,2023-11-14 05:36:44,0
